In [ ]:
import zipfile, os

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

In [ ]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
batch_size = 15
nb_train_samples = 20000
nb_validation_samples = 5000

nb_images = 25000
nb_classes = 2
nb_class_images = nb_images // nb_classes

val_data_portion = 0.2

In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "cats"))
    os.makedirs(os.path.join(dir_name, "dogs"))

In [ ]:
data_dir = '/kaggle/working/train'
train_dir = '/kaggle/working/set/train' 
val_dir = '/kaggle/working/set/val' 

In [ ]:
create_directory(train_dir)
create_directory(val_dir)

In [ ]:
import shutil

def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "cat." + str(i) + ".jpg"), os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir, "dog." + str(i) + ".jpg"), os.path.join(dest_dir, "dogs"))

In [ ]:
start_val_data_idx = int(nb_class_images * (1-val_data_portion))


copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, 12500, data_dir, val_dir)

In [ ]:
print('Cats in train set: ', len(os.listdir(train_dir + '/cats')))
print('Dogs in train set: ', len(os.listdir(train_dir + '/dogs')), '\n')

print('Cats in validation set: ', len(os.listdir(val_dir + '/cats')))
print('Cats in validation set: ', len(os.listdir(val_dir + '/dogs')), '\n')

In [ ]:
from tensorflow import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu' input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
 optimizer='adam',
 metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
 train_dir,
 target_size=(img_width, img_height),
 batch_size=batch_size,
 class_mode='binary')


In [ ]:
val_generator = datagen.flow_from_directory(
 val_dir,
 target_size=(img_width, img_height),
 batch_size=batch_size,
 class_mode='binary')

In [ ]:
history = model.fit_generator(
 train_generator,
 steps_per_epoch=nb_train_samples // batch_size,
 epochs=10,
 validation_data=val_generator,
 validation_steps=nb_validation_samples // batch_size)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Train_loss")
ax1.plot(history.history['val_loss'], color='r', label="Val_loss")
ax1.set_xticks(np.arange(1, 10, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Train_accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Val_accuracy")
ax2.set_xticks(np.arange(1, 10, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()